<a href="https://colab.research.google.com/github/gdollp/mlb/blob/main/lgbm_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook uses lightGBM to make predictions.

We use the following features
* playerId
* position
* teamId(rosters)
* status(rosters)
* playerBoxScores

and the date 20200401~20200431 as the validation data.

But I think there is room for improvement.  
If you have better ways, I would appreciate it if you could comment on it.

このnotebookではlightGBMを使って予測します。

特徴量は以下のものを使用しています。
* playerId
* position
* teamId(rosters)
* status(rosters)
* playerBoxScores

20200401~20200431を日時をvalidation dataとしていますが、一考の余地がありそうです。  
もし良さそうな方法があればコメントしていただけると幸いです。



In [174]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [175]:
#gamesを特徴量に追加


#基本指針 #新しい特徴量を考える　or 利用できていないデータセットの利用

## About Dataset

Train.csv is stored as a csv file with each column as follows.  

train.csvを以下のようにして各カラムをcsvファイルとして保管しています。

In [176]:
%%capture
"""
!pip install pandarallel 

import gc

import numpy as np
import pandas as pd
from pathlib import Path

from pandarallel import pandarallel
pandarallel.initialize()

BASE_DIR = Path('../input/mlb-player-digital-engagement-forecasting')
train = pd.read_csv(BASE_DIR / 'train.csv')

null = np.nan
true = True
false = False

for col in train.columns:

    if col == 'date': continue

    _index = train[col].notnull()
    train.loc[_index, col] = train.loc[_index, col].parallel_apply(lambda x: eval(x))

    outputs = []
    for index, date, record in train.loc[_index, ['date', col]].itertuples():
        _df = pd.DataFrame(record)
        _df['index'] = index
        _df['date'] = date
        outputs.append(_df)

    outputs = pd.concat(outputs).reset_index(drop=True)

    outputs.to_csv(f'{col}_train.csv', index=False)
    outputs.to_pickle(f'{col}_train.pkl')

    del outputs
    del train[col]
    gc.collect()
"""

## Training

In [177]:
#!pip install optuna

In [178]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error
from datetime import timedelta
from functools import reduce
from tqdm import tqdm
#import optuna.integration.lightgbm as lgbm
import lightgbm as lgbm
import pickle
#import mlb
import sklearn #機械学習のライブラリ
from sklearn.decomposition import PCA #主成分分析器
from sklearn.preprocessing import LabelEncoder
import gc
from datetime import datetime

In [179]:
BASE_DIR = Path('/content/drive/MyDrive/mlb/input')
TRAIN_DIR = Path('/content/drive/MyDrive/mlb/input/archive')

In [180]:
players = pd.read_csv(BASE_DIR / 'players.csv')

rosters = pd.read_pickle(TRAIN_DIR / 'rosters_train.pkl')
targets = pd.read_pickle(TRAIN_DIR / 'nextDayPlayerEngagement_train.pkl')
scores1 = pd.read_pickle(TRAIN_DIR / 'playerBoxScores_train.pkl')
scores = scores1.groupby(['playerId', 'date']).sum().reset_index()
playertwitter = pd.read_pickle("/content/drive/MyDrive/mlb/input/archive/playerTwitterFollowers_train.pkl")
games = pd.read_pickle(TRAIN_DIR / 'games_train.pkl')
events = pd.read_pickle(TRAIN_DIR / 'events_train.pkl')
standings = pd.read_pickle(TRAIN_DIR / 'standings_train.pkl')
teamtwitter = pd.read_pickle(TRAIN_DIR / 'teamTwitterFollowers_train.pkl')
transaction = pd.read_pickle(TRAIN_DIR / 'teamTwitterFollowers_train.pkl')
awards = pd.read_csv(BASE_DIR / 'awards.csv')
seasons = pd.read_csv(BASE_DIR / 'seasons.csv')
teams = pd.read_csv(BASE_DIR / 'teams.csv')
player_target_stats = pd.read_csv(BASE_DIR /"player_target_stats.csv")


In [181]:
events_p = events
events_p["playerId"] = events_p["pitcherId"]
events_h = events
events_h["playerId"] = events_p["hitterId"]

In [192]:
teamtwitter

,date,teamId,teamName,accountName,twitterHandle,numberOfFollowers,index
0,20180101,147,New York Yankees,New York Yankees,@Yankees,3130482,0
1,20180101,112,Chicago Cubs,Chicago Cubs,@Cubs,2373710,0
2,20180101,141,Toronto Blue Jays,Toronto Blue Jays,@BlueJays,2196352,0
3,20180101,111,Boston Red Sox,Boston Red Sox,@RedSox,1950737,0
4,20180101,119,Los Angeles Dodgers,Los Angeles Dodgers,@Dodgers,1949542,0
...,...,...,...,...,...,...,...
1195,20210401,115,Colorado Rockies,Colorado Rockies,@Rockies,585102,1186
1196,20210401,133,Oakland Athletics,Oakland Athletics,@Athletics,572565,1186
1197,20210401,136,Seattle Mariners,Seattle Mariners,@Mariners,571143,1186
1198,20210401,135,San Diego Padres,San Diego Padres,@Padres,466044,1186


In [183]:
awards2 = awards.groupby("playerId").count()
awards2 = awards2.reset_index()
awards2["awardName"]

0       12
1       38
2       24
3       43
4       20
        ..
1691     1
1692     2
1693     1
1694     1
1695     1
Name: awardName, Length: 1696, dtype: int64

In [184]:
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status', 'date']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', 'date',"gamePk"]
games_cols = ["gamePk","homeId","dayNight","seriesDescription","gamesInSeries","homeWinner","awayWinner","homeScore","awayScore","gameType"]
playertwitter_cols = ["date","playerId","numberOfFollowers"]
awards_cols = ["playerId","awardName"]
standings_cols = ["date","teamId","divisionRank","divisionLeader","wildCardLeader","leagueRank","divisionId","gameDate"]
teamtwitter_cols = ["date","teamId","numberOfFollowers"]

feature_cols1 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',"divisionId",
       'assists', 'putOuts', 'errors', 'chances',"ishome","label_seriesDescription","gamesInSeries","label_daynight","gameType"]#winorlose"]


feature_cols2 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',"wildCardLeader",
       'assists', 'putOuts', 'errors', 'chances',"ishome","label_seriesDescription","gamesInSeries","label_daynight","gameType","awardName",'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max','target2_prob',"divisionRank","divisionId"]#winorlose"]

feature_cols3 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',"wildCardLeader",
       'assists', 'putOuts', 'errors', 'chances',"ishome","label_seriesDescription","gamesInSeries","label_daynight","gameType","awardName",
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',"divisionRank","divisionId"
 ]

       #winorlose"]

feature_cols4 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances','target4_mean',"gameType","awardName","divisionId",
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',"divisionRank"]

In [185]:
# creat dataset
train = targets[targets_cols].merge(players[players_cols], on=['playerId'], how='left')
train = train.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')
train = train.merge(scores[scores_cols], on=['playerId', 'date'], how='left')
train = train.merge(games[games_cols], on=["gamePk"], how="left")
train = train.merge(playertwitter[playertwitter_cols], on=['playerId', 'date'], how='left')
#train = train.merge(events[events_cols], on=['gamePk', 'date'], how='left')
train = train.merge(awards2[awards_cols], on=['playerId'], how='left')
train = train.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])
train = train.merge(standings[standings_cols], on=['date',"teamId"], how='left')
train = train.merge(standings[standings_cols], on=['date',"teamId"], how='left')

In [186]:
train["ongame"] = np.where(train["gamePk"].isnull() == 1,0,1)
train["ongame"].unique()
train["ishome"] = ""
train["ishome"] = np.where(train["teamId"]==train["homeId"],2,train["ongame"])
train["winorlose"] = np.where(train["teamId"]==train["homeId"],train["homeWinner"],train["awayWinner"])
train["winorlose"] = train["winorlose"].fillna(2.0).astype(int)
train["winorlose"].unique()
train["score"] = np.where(train["teamId"]==train["homeId"],train["homeScore"],train["awayScore"])
#train["bestmoment"] =np.where(train["playerId"] == 543333,1,0 )

#"bestmoment","wsmvp",
train["divisionRank"] = train["divisionRank"].fillna(7.0).astype(int)
train["divisionLeader"] = train["divisionLeader"].fillna(-1.0).astype(int)
train["wildCardLeader"] = np.where(train["wildCardLeader"]=="True",1,0)
train["leagueRank"]  = train["leagueRank"].fillna(-1.0).astype(int)

In [190]:
train["pddate"] = pd.to_datetime(train['gameDate'],format='%Y-%m-%d')

train["youbi"] = train["pddate"].dt.dayofweek
train["youbi"].unique()

array([nan,  3.,  4.,  5.,  6.,  0.,  1.,  2.])

In [191]:
train["youbi"].isnull().sum()

1844643

In [160]:
train[["youbi","date"]].to_csv("/content/drive/MyDrive/mlb/input/youbi.csv")

In [161]:


# label encoding
player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
daynight2num = {c: i for i, c in enumerate(train['dayNight'].unique())}
seriesDescription2num = {c: i for i, c in enumerate(train['seriesDescription'].unique())}
gameType2num = {c: i for i, c in enumerate(train['gameType'].unique())}
#awardName2num = {c: i for i, c in enumerate(train['awardName'].unique())}
train['label_playerId'] = train['playerId'].map(player2num)
train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)
train["label_daynight"] = train['dayNight'].map(daynight2num)
train["label_seriesDescription"] = train["seriesDescription"].map(seriesDescription2num)
train["gameType"] = train["gameType"].map(gameType2num)
#train["awardName"] = train["awardName"].map(awardName2num)


In [161]:
df["gameDate"]

In [162]:
#make feature


In [163]:
train_X1 = train[feature_cols1]
train_X2 = train[feature_cols2]
train_X3 = train[feature_cols3]
train_X4 = train[feature_cols4]

"""train_data = [train_X1,train_X2,train_X3,train_X4]

remove1 = []
remove2 = []
remove3 = []
remove4 = []
remove_col = [remove1,remove2,remove3,remove4]"""

train_y = train[['target1', 'target2', 'target3', 'target4']]
"""
for i in range(len(train_data)):
  for col in train_data[i].columns:
    if train_data[i][col].std() == 0:
        remove_col[i].append(col)
        train_data[i].drop(remove_col[i], axis=1, inplace=True) 
"""

'\nfor i in range(len(train_data)):\n  for col in train_data[i].columns:\n    if train_data[i][col].std() == 0:\n        remove_col[i].append(col)\n        train_data[i].drop(remove_col[i], axis=1, inplace=True) \n'

In [164]:
params1 = {'objective':'mae','reg_alpha': 0.14947461820098767, 'reg_lambda': 0.10185644384043743, 'n_estimators': 3633, 'learning_rate': 0.08046301304430488, 'num_leaves': 674, 'feature_fraction': 0.9101240539122566, 'bagging_fraction': 0.9884451442950513, 'bagging_freq': 8, 'min_child_samples': 51}

params2 = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 80,
 'learning_rate': 0.1,
 'random_state': 42,
 "num_leaves": 22
}

params4 = {'objective':'mae','reg_alpha': 0.016468100279441976, 'reg_lambda': 0.09128335764019105, 'n_estimators': 9868, 'learning_rate': 0.10528150510326864, 'num_leaves': 157, 'feature_fraction': 0.5419185713426886, 'bagging_fraction': 0.2637405128936662, 'bagging_freq': 19, 'min_child_samples': 71}

"""
params = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 10000,
 'learning_rate': 0.1,
 'random_state': 42,
 "num_leaves": 100
}"""

In [165]:
#train.to_csv("/content/drive/MyDrive/mlb/script/output/train.csv")


_index = (train['date'] < 20210401)

del train
gc.collect()


x_train1 = train_X1.loc[_index].reset_index(drop=True)
y_train = train_y.loc[_index].reset_index(drop=True)
x_valid1 = train_X1.loc[~_index].reset_index(drop=True)
y_valid = train_y.loc[~_index].reset_index(drop=True)

x_train2 = train_X2.loc[_index].reset_index(drop=True)
x_valid2 = train_X2.loc[~_index].reset_index(drop=True)
x_train3 = train_X3.loc[_index].reset_index(drop=True)
x_valid3 = train_X3.loc[~_index].reset_index(drop=True)
x_train4 = train_X4.loc[_index].reset_index(drop=True)
x_valid4 = train_X4.loc[~_index].reset_index(drop=True)

del train_X1,train_X2,train_X3,train_X4
gc.collect()

0

In [166]:
def fit_lgbm(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    #model = lgbm.LGBMRegressor(**params)
    best_params, tuning_history = {}, []
    
    lgb_train = lgbm.Dataset(x_train, y_train)
    lgb_eval = lgbm.Dataset(x_valid, y_valid, reference=lgb_train)
    model = lgbm.train(params, 
        lgb_train, valid_sets=lgb_eval,  
        early_stopping_rounds=100,
        num_boost_round=500,
        verbose_eval=100,
        #verbose=verbose,               
        )
    oof_pred = model.predict(x_valid)
    score = mean_absolute_error(oof_pred, y_valid)
    print('mae:', score)
    return oof_pred, model, score


# training lightgbm
params = {
    'boosting_type': 'gbdt',
    'objective': 'mae',
    'metric': "mae",
    'reg_alpha': 0.1,
    'reg_lambda': 0.1, 
    'learning_rate': 0.1,
    "num_leaves": 100,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

oof1, model1, score1 = fit_lgbm(
    x_train1, y_train['target1'],
    x_valid1, y_valid['target1'],
    params1
)
file = '/content/drive/MyDrive/mlb/script/output/model1.pkl'
pickle.dump(model1, open(file, 'wb'))

oof2, model2, score2 = fit_lgbm(
    x_train2, y_train['target2'],
    x_valid2, y_valid['target2'],
    params2
)
file = '/content/drive/MyDrive/mlb/script/output/model2.pkl'
pickle.dump(model2, open(file, 'wb'))
oof3, model3, score3 = fit_lgbm(
    x_train3, y_train['target3'],
    x_valid3, y_valid['target3'],
    params
)
file = '/content/drive/MyDrive/mlb/script/output/model3.pkl'
pickle.dump(model3, open(file, 'wb'))
oof4, model4, score4 = fit_lgbm(
    x_train4, y_train['target4'],
    x_valid4, y_valid['target4'],
    params4
)
file = '/content/drive/MyDrive/mlb/script/output/model4.pkl'
pickle.dump(model4, open(file, 'wb'))
score = (score1+score2+score3+score4) / 4
print(f'score: {score}')

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 0.693506
[200]	valid_0's l1: 0.688475
[300]	valid_0's l1: 0.686385
[400]	valid_0's l1: 0.685342
[500]	valid_0's l1: 0.683786
[600]	valid_0's l1: 0.682605
[700]	valid_0's l1: 0.682316
[800]	valid_0's l1: 0.682095
[900]	valid_0's l1: 0.682007
Early stopping, best iteration is:
[853]	valid_0's l1: 0.681929
mae: 0.6819292213005095


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[76]	valid_0's l1: 1.41728
mae: 1.417275163352289


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 0.551972
[200]	valid_0's l1: 0.551696
[300]	valid_0's l1: 0.551518
Early stopping, best iteration is:
[252]	valid_0's l1: 0.551513
mae: 0.5515126965244794


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 1.12705
[200]	valid_0's l1: 1.12405
[300]	valid_0's l1: 1.12364
[400]	valid_0's l1: 1.12296
[500]	valid_0's l1: 1.12142
[600]	valid_0's l1: 1.11929
[700]	valid_0's l1: 1.11925
[800]	valid_0's l1: 1.11905
Early stopping, best iteration is:
[730]	valid_0's l1: 1.11893
mae: 1.1189314759450308
score: 0.9424121392805773


In [167]:
importance = pd.DataFrame(model1.feature_importance(), index=x_train1.columns, columns=['importance'])
pd.set_option('display.max_rows', 500)
importance

,importance
label_playerId,330891
label_primaryPositionName,72948
label_teamId,86701
label_status,19575
battingOrder,3518
gamesPlayedBatting,44
flyOuts,435
groundOuts,726
runsScored,448
doubles,211


In [168]:
model1.params

{'bagging_fraction': 0.9884451442950513,
 'bagging_freq': 8,
 'feature_fraction': 0.9101240539122566,
 'learning_rate': 0.08046301304430488,
 'min_child_samples': 51,
 'num_leaves': 674,
 'objective': 'mae',
 'reg_alpha': 0.14947461820098767,
 'reg_lambda': 0.10185644384043743}

In [169]:
model2.params

{'learning_rate': 0.1,
 'num_leaves': 22,
 'objective': 'mae',
 'random_state': 42,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1}

In [170]:
model3.params

{'learning_rate': 0.1,
 'num_leaves': 100,
 'objective': 'mae',
 'random_state': 42,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1}

In [171]:
model4.params

{'bagging_fraction': 0.2637405128936662,
 'bagging_freq': 19,
 'feature_fraction': 0.5419185713426886,
 'learning_rate': 0.10528150510326864,
 'min_child_samples': 71,
 'num_leaves': 157,
 'objective': 'mae',
 'reg_alpha': 0.016468100279441976,
 'reg_lambda': 0.09128335764019105}

## Inference

In [172]:
"""players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']

null = np.nan
true = True
false = False

env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in iter_test: # make predictions here
    
    sample_prediction_df = sample_prediction_df.reset_index(drop=True)
    
    # creat dataset
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[1]))
    # Dealing with missing values
    if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
        test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
    else:
        test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in rosters.columns:
            if col == 'playerId': continue
            test_rosters[col] = np.nan
            
    if test_df['playerBoxScores'].iloc[0] == test_df['playerBoxScores'].iloc[0]:
        test_scores = pd.DataFrame(eval(test_df['playerBoxScores'].iloc[0]))
    else:
        test_scores = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in scores.columns:
            if col == 'playerId': continue
            test_scores[col] = np.nan
    test_scores = test_scores.groupby('playerId').sum().reset_index()
    test = sample_prediction_df[['playerId']].copy()
    test = test.merge(players[players_cols], on='playerId', how='left')
    test = test.merge(test_rosters[rosters_cols], on='playerId', how='left')
    test = test.merge(test_scores[scores_cols], on='playerId', how='left')

    test['label_playerId'] = test['playerId'].map(player2num)
    test['label_primaryPositionName'] = test['primaryPositionName'].map(position2num)
    test['label_teamId'] = test['teamId'].map(teamid2num)
    test['label_status'] = test['status'].map(status2num)
    
    test_X = test[feature_cols]
    
    # predict
    pred1 = model1.predict(test_X)
    pred2 = model2.predict(test_X)
    pred3 = model3.predict(test_X)
    pred4 = model4.predict(test_X)
    
    # merge submission
    sample_prediction_df['target1'] = np.clip(pred1, 0, 100)
    sample_prediction_df['target2'] = np.clip(pred2, 0, 100)
    sample_prediction_df['target3'] = np.clip(pred3, 0, 100)
    sample_prediction_df['target4'] = np.clip(pred4, 0, 100)
    sample_prediction_df = sample_prediction_df.fillna(0.)
    del sample_prediction_df['playerId']
    
    env.predict(sample_prediction_df)"""

"players_cols = ['playerId', 'primaryPositionName']\nrosters_cols = ['playerId', 'teamId', 'status']\nscores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',\n       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',\n       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',\n       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',\n       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',\n       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',\n       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',\n       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',\n       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',\n       'groundOutsPitching', 'runsPitching', 'doublesPitching',\n       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',\n       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',\n       'hitByPitchPitching', 'atBatsPitching'